In [1]:
# cycles_to_algebra.py
# --------------- 
# 生成 12 条最短简单回路的代数占位符表示（每条只给出“自我贡献”那一部分）
# 约定：x0..x11 表示 12 个输入块；AES(s) 表示对 s 做 AES-like 映射；^ 表示 XOR（字符串占位）
#
# 说明：XOR 边在沿路传播时不改变“贡献表达式”的形式（线性的传播），所以我们只在遇到 AES 边时
# 把传播值包装为 AES(...)。最终的表达式表示从起点 x_i 沿该回路回来，对自身产生的贡献。
# 若想得到完整的 y_i（块 i 的输出），还需把其它进入 i 的直接贡献（其它节点对 i 的边）累加上去。

def AES(s):
    return f"AES({s})"

# 我们把之前找到的每个节点的最短回路写成列表 (0-based node indices) 以及对应 edge ops
# （ops 与节点列表一一对应，ops[i] 是从 nodes[i] -> nodes[i+1] 的边的操作）
shortest_cycles = {
    0: {"nodes": [0,1,3,2,0], "ops": ["XOR","AES","AES","AES"]},
    1: {"nodes": [1,3,2,0,1], "ops": ["AES","AES","AES","XOR"]},
    2: {"nodes": [2,0,1,3,2], "ops": ["AES","XOR","AES","AES"]},
    3: {"nodes": [3,6,4,3],   "ops": ["XOR","XOR","XOR"]},
    4: {"nodes": [4,3,6,4],   "ops": ["XOR","XOR","XOR"]},
    5: {"nodes": [5,7,5],     "ops": ["XOR","XOR"]},
    6: {"nodes": [6,4,3,6],   "ops": ["XOR","XOR","XOR"]},
    7: {"nodes": [7,5,7],     "ops": ["XOR","XOR"]},
    8: {"nodes": [8,5,11,4,8],"ops": ["XOR","XOR","XOR","XOR"]},
    9: {"nodes": [9,10,2,0,9],"ops": ["XOR","XOR","AES","AES"]},
    10:{"nodes": [10,2,0,9,10],"ops":["XOR","AES","AES","XOR"]},
    11:{"nodes": [11,4,8,5,11],"ops":["XOR","XOR","XOR","XOR"]}
}

# 生成每条回路的“自我贡献表达式”
expressions = {}
for start, info in shortest_cycles.items():
    nodes = info["nodes"]
    ops = info["ops"]
    # initial contribution is the starting block variable
    v = f"x{start}"
    # walk along edges in order and apply AES when edge op is AES; XOR edges do not change the carried contribution form
    for op in ops:
        if op.upper() == "AES":
            v = AES(v)
        elif op.upper() == "XOR":
            # XOR edges propagate the contribution linearly: keep v unchanged as "contribution"
            # (we keep it as same symbol v)
            pass
        else:
            raise ValueError("unknown op: " + str(op))
    # v is the contribution (string) that starting x{start} produces back to itself via this loop
    expressions[start] = {
        "cycle_nodes_0based": nodes,
        "cycle_nodes_1based": [i+1 for i in nodes],
        "ops": ops,
        "self_contribution": v,
        "comment": "This is the contribution of x{0} propagated around the loop back to itself.".format(start)
    }

# 打印友好格式
for i in range(12):
    e = expressions[i]
    print(f"Block {i} (1-based {i+1}):")
    print("  Path (0-based):", e["cycle_nodes_0based"])
    print("  Path (1-based):", e["cycle_nodes_1based"])
    print("  Ops:", e["ops"])
    print("  Self-contribution expression:", e["self_contribution"])
    print()

# 示例输出说明：
# - 假如 block 2 的最短回路是 [2,0,1,3,2] 且 ops = [AES,XOR,AES,AES]，
#   则代码会输出 self-contribution = AES(AES(AES(x2))) （因为遇到三个 AES 边）
#
# 扩展（若要得到完整 y_i）：
# - 在每一条路径走到某个中间节点 j 时，该节点 j 可能还接受来自其它输入的边（直接 XOR / AES）。
# - 要把 y_i 写成完整的代数式，需要把所有进入 i 的贡献（包括这条回路的自我贡献和其它节点直接对 i 的贡献）
#   全部累加（XOR）。如果你需要，我可以把“每个节点的所有直接进入贡献”也一并列出并构成完整 y_i 的占位表达式。


Block 0 (1-based 1):
  Path (0-based): [0, 1, 3, 2, 0]
  Path (1-based): [1, 2, 4, 3, 1]
  Ops: ['XOR', 'AES', 'AES', 'AES']
  Self-contribution expression: AES(AES(AES(x0)))

Block 1 (1-based 2):
  Path (0-based): [1, 3, 2, 0, 1]
  Path (1-based): [2, 4, 3, 1, 2]
  Ops: ['AES', 'AES', 'AES', 'XOR']
  Self-contribution expression: AES(AES(AES(x1)))

Block 2 (1-based 3):
  Path (0-based): [2, 0, 1, 3, 2]
  Path (1-based): [3, 1, 2, 4, 3]
  Ops: ['AES', 'XOR', 'AES', 'AES']
  Self-contribution expression: AES(AES(AES(x2)))

Block 3 (1-based 4):
  Path (0-based): [3, 6, 4, 3]
  Path (1-based): [4, 7, 5, 4]
  Ops: ['XOR', 'XOR', 'XOR']
  Self-contribution expression: x3

Block 4 (1-based 5):
  Path (0-based): [4, 3, 6, 4]
  Path (1-based): [5, 4, 7, 5]
  Ops: ['XOR', 'XOR', 'XOR']
  Self-contribution expression: x4

Block 5 (1-based 6):
  Path (0-based): [5, 7, 5]
  Path (1-based): [6, 8, 6]
  Ops: ['XOR', 'XOR']
  Self-contribution expression: x5

Block 6 (1-based 7):
  Path (0-based): [6

In [2]:
import numpy as np

op_matrix = np.array([
    [0,0,2,0,0,0,0,0,1,0,0,0],
    [1,0,0,0,0,0,0,0,0,1,0,0],
    [0,0,0,2,0,0,0,0,0,0,1,0],
    [0,2,0,0,1,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,1,0,0,0,0,1],
    [0,0,0,0,0,0,0,1,1,0,0,0],
    [0,0,1,1,0,0,0,0,0,0,0,0],
    [0,1,0,0,0,1,0,0,0,0,0,0],
    [0,0,0,0,1,0,2,0,0,0,0,0],
    [2,0,0,0,0,0,0,0,0,0,0,0],
    [0,0,0,0,0,0,0,2,0,1,0,0],
    [0,0,0,0,0,1,0,0,0,0,1,0]
], dtype=int)

# 寻找最短自环路径（BFS）
from collections import deque

def shortest_self_loop(start):
    n = len(op_matrix)
    q = deque([(start, [start])])
    visited = set([start])
    while q:
        node, path = q.popleft()
        for nxt in range(n):
            if op_matrix[node, nxt] != 0:
                if nxt == start and len(path) > 1:
                    return path + [start]
                if nxt not in path:  # simple path
                    q.append((nxt, path + [nxt]))
    return None

# 把路径翻译成代数表达式
def path_to_expr(path):
    expr = f"x{path[0]}"
    for i in range(len(path)-1):
        u, v = path[i], path[i+1]
        if op_matrix[u, v] == 2:  # AES
            expr = f"AES({expr})"
        elif op_matrix[u, v] == 1:  # XOR
            expr = f"({expr} + x{v})"
    return expr

# 生成每个块的表达式
expressions = {}
for i in range(12):
    path = shortest_self_loop(i)
    if path:
        expressions[f"x{i}"] = path_to_expr(path)

# 打印
for k, v in expressions.items():
    print(f"{k} = {v}")


x0 = (AES(AES(AES(x0))) + x0)
x1 = AES(AES(AES((x1 + x0))))
x2 = AES((AES(AES(x2)) + x0))
x3 = (((x3 + x4) + x6) + x3)
x4 = (((x4 + x6) + x3) + x4)
x5 = ((x5 + x7) + x5)
x6 = (((x6 + x3) + x4) + x6)
x7 = ((x7 + x5) + x7)
x8 = ((((x8 + x4) + x11) + x5) + x8)
x9 = ((AES(AES(x9)) + x10) + x9)
x10 = (AES(AES((x10 + x9))) + x10)
x11 = ((((x11 + x5) + x8) + x4) + x11)
